# Day 16: The Floor Will Be Lava

[*Advent of Code 2023 day 16*](https://adventofcode.com/2023/day/16) and [*solution megathread*](https://redd.it/18jjpfk)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/16/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F16%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
part1_example_input = '''.|...\\....
|.-.\\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|....'''

In [5]:
from enum import Enum

class Cardinal(Enum):
    EAST = (0, 1)
    NORTH = (-1, 0)
    WEST = (0, -1)
    SOUTH = (1, 0)
    
    def __str__(self):
            match self:
                case Cardinal.EAST:
                    return '>'
                case Cardinal.NORTH:
                    return '^'
                case Cardinal.WEST:
                    return '<'
                case Cardinal.SOUTH:
                    return 'v'
                case _:
                    return None

print(str(Cardinal.EAST))

>


In [6]:
from functools import cache


@cache
def process_beam(mirrors, location, heading):
    def progress_beam(location, heading):
        new_location = (
            location[0] + heading.value[0],
            location[1] + heading.value[1]
            )
        if all(0 <= l < len(mirrors) for l in new_location):
            return new_location
        else:
            return None


    match mirrors[location[0]][location[1]]:
        case '/':
            match heading:
                case Cardinal.EAST:
                    heading = Cardinal.NORTH
                case Cardinal.NORTH:
                    heading = Cardinal.EAST
                case Cardinal.WEST:
                    heading = Cardinal.SOUTH
                case Cardinal.SOUTH:
                    heading = Cardinal.WEST
                case _:
                    pass
            if (new_location := progress_beam(location, heading)):
                return [(new_location, heading)]
        case '\\':
            match heading:
                case Cardinal.EAST:
                    heading = Cardinal.SOUTH
                case Cardinal.NORTH:
                    heading = Cardinal.WEST
                case Cardinal.WEST:
                    heading = Cardinal.NORTH
                case Cardinal.SOUTH:
                    heading = Cardinal.EAST
                case _:
                    pass
            if (new_location := progress_beam(location, heading)):
                return [(new_location, heading)]
        case '-':
            match heading:
                case Cardinal.NORTH | Cardinal.SOUTH:
                    results = []
                    heading = Cardinal.WEST
                    if (new_location := progress_beam(location, heading)):
                        results.append((new_location, heading))
                    heading = Cardinal.EAST
                    if (new_location := progress_beam(location, heading)):
                        results.append((new_location, heading))
                    return results
                case _:
                    pass
        case '|':
            match heading:
                case Cardinal.EAST | Cardinal.WEST:
                    results = []
                    heading = Cardinal.SOUTH
                    if (new_location := progress_beam(location, heading)):
                        results.append((new_location, heading))
                    heading = Cardinal.NORTH
                    if (new_location := progress_beam(location, heading)):
                        results.append((new_location, heading))
                    return results
                case _:
                    pass
        case _:
            pass

    if (new_location := progress_beam(location, heading)):
        return [(new_location, heading)]

In [7]:
mirrors = tuple(part1_example_input.splitlines())
# mirrors = tuple(downloaded['input'].splitlines())

# @cache - beam cycles would prevent me from currently caching 
#          the entire path a beam energize
def expose(mirrors, origin_location, origin_heading):
    traveling_light = [(origin_location, origin_heading)]
    energized = {origin_location: [origin_heading]}

    while traveling_light:
        location, heading = traveling_light.pop()
        # print(f"{location=}, {heading=}, {traveling_light=}")
        if (new_beams := process_beam(mirrors, location, heading)):
            for new_location, new_heading in new_beams:
                if (new_location not in energized.keys() or
                        new_heading not in energized[new_location]):
                    if new_location not in energized.keys():
                        energized[new_location] = [new_heading]
                    else:
                        energized[new_location].append(new_heading)
                    traveling_light.append((new_location, new_heading))
    return energized


energized = expose(mirrors, (0, 0), Cardinal.EAST)

In [8]:
from itertools import count


def superimpose_beams(mirrors, energized):
    def arrow(card):
        if len(card) == 1:
            return str(card[0])
        else:
            return str(len(card))
    
    
    results = []
    for row, line in zip(count(), mirrors):
        super_line = []
        for col, l in zip(count(), line):
            if l != '.':
                super_line.append(l)
            else:
                if (row, col) in energized.keys():
                    super_line.append(arrow(energized[(row, col)]))
                else:
                    super_line.append('.')
        results.append(''.join(super_line))
    return '\n'.join(results)

print(superimpose_beams(mirrors, energized))
print(len(energized))
# >|<<<\....
# |v-.\^....
# .v...|->>>
# .v...v^.|.
# .v...v^...
# .v...v^..\
# .v../2\\..
# <->-/vv|..
# .|<<<2-|.\
# .v//.|.v..

>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
46


In [9]:
HTML(downloaded['part2'])

In [10]:
results = {}

row = 0
heading = Cardinal.SOUTH
for col in range(0,len(mirrors)):
    results[(row, col, heading)] = len(expose(mirrors, (row, col), heading))
row = len(mirrors) - 1
heading = Cardinal.NORTH
for col in range(0,len(mirrors)):
    results[(row, col, heading)] = len(expose(mirrors, (row, col), heading))
col = 0
heading = Cardinal.EAST
for row in range(0,len(mirrors)):
    results[(row, col, heading)] = len(expose(mirrors, (row, col), heading))
col = len(mirrors) - 1
heading = Cardinal.WEST
for row in range(0,len(mirrors)):
    results[(row, col, heading)] = len(expose(mirrors, (row, col), heading))

print(best := max(results, key=results.get))
print(results[best])

(0, 3, <Cardinal.SOUTH: (1, 0)>)
51
